# Agenda: Dates and times in Pandas!

1. Dates and times in programming
2. Parsing dates with Pandas
3. The `.dt` accessor
4. Loading multiple files into a single data frame
5. How can you parse odd date formats?
6. Grouping with dates
7. Time deltas
8. Time series
9. Resampling

# Dates and times in programming

The most natural (and reasonable) way to to think about dates and times is as specific, unique moments in time. I can identify a "datetime" at various points in my life, and each of these is unique:

- When I was born
- When I graduated from university
- When this meeting started
- When this meeting ends

In programming, we describe these as "datetime" or "timestamp" objects.

We also talk about spans of time. Here, there is no hour/minute/second or year/month/day. Rather, it's "10 minutes long" or "3 days long."  Some examples:

- This meeting will last for 1 hour
- My life, so far, has been about 51.5 years.
- The pandemic has been going on for about 2 years now

This kind of measurement is known as a "time delta," or an "interval."

You can do some basic date+time math:

- timestamp + interval = timestamp
- timestamp - timestamp = interval

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

filename = '../data/nyc_taxi_2019-01.csv'
df = pd.read_csv(filename, 
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count', 'total_amount', 'trip_distance'])


In [2]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55


In [3]:
df.dtypes

tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
total_amount             float64
dtype: object